In [33]:
from hyperopt import fmin, STATUS_OK, Trials, hp, tpe, rand
import lightgbm
import pandas as pd 
import numpy as np
from pathlib import Path
import os 
import sys 

home_dir = '/Users/Liu'
scripts_dir = os.path.join(home_dir, 'NBA_Pro_Line_Analytics/GBM_model_build')
sys.path.insert(1, scripts_dir)

import model_build_scripts
from model_build_scripts import helpers
from model_build_scripts.NBA_model_build import NBA_Model_Build

In [34]:
#Defines the model scenario in ./configs/scenarios_config.yml
model_name = "GBM_ht_cover_spread_outcome_model_build_all_features"

In [35]:
home_dir = Path(home_dir)
scenario_dir = home_dir / 'NBA_Pro_Line_Analytics' / 'GBM_model_build' / 'scenarios_config.yml'
feature_dir = home_dir / 'NBA_Pro_Line_Analytics' / 'GBM_model_build' / 'features_config.yml'
model_scenario = NBA_Model_Build(model_name, 
                                 feature_yaml = feature_dir, 
                                 scenario_yaml = scenario_dir)

/Users/Liu/NBA_Pro_Line_Analytics/GBM_model_build/model_build_scripts/helpers.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(stream)


In [36]:
model_scenario.load_data()

In [103]:
initial_params = {'objective': 'binary',
                  'metric': 'auc',
                  'num_boost_round': 1000,
                  'early_stopping_rounds': 20,
                  'max_depth': 15,
                  'seed': model_scenario.seed,
                  'bagging_freq': 1
                 }

In [104]:
#Runs Recursive Feature Elimination to Remove all features of 0 Importance
post_rfe_model, var_importance  = model_scenario.run_rfe(model_params = initial_params, 
                                                         target = model_scenario.target, X_vars= model_scenario.X_vars)

761 features below threshold
The following features will be removed:
['Num_Players_15plus_APG_L3G_HT', 'Num_Players_15plus_PPG_L3G_RT', 'RT_cnt_within_5_L8G', 'Num_Players_10plus_TOT_LG_RT', 'HT_Num_Losses_L4G', 'TS_PCT_L10G_RT', 'DEFF_L6G_RT', 'Num_Players_3plus_TO_L4G_HT', 'RT_cnt_wins_6_plus_L7G', 'Num_Players_1plus_BL_L2G_RT', 'Num_Players_30plus_PPG_L6G_HT', 'RT_num_11_plus_pts_loss_LG', 'Num_Players_10plus_TOT_L6G_HT', 'Num_Players_3plus_BL_L6G_HT', 'POSS_LG_HT', 'FT_PCT_L2G_HT', 'RT_cnt_wins_6_plus_L6G', 'Num_Players_3plus_ST_L9G_RT', 'Num_Players_5plus_APG_L8G_HT', 'Num_Players_5plus_APG_L7G_RT', 'TS_PCT_LG_HT', 'Num_Players_15plus_PPG_L6G_HT', 'POSS_L10G_RT', 'HT_num_11_plus_pts_wins_L9G', 'TS_PCT_L6G_RT', 'HT_num_11_plus_pts_wins_L10G', 'PACE_L7G_RT', 'Num_Players_2plus_ST_L5G_HT', '3PT_PCT_L5G_HT', 'RT_num_11_plus_pts_wins_L9G', 'Num_Players_2plus_ST_LG_HT', 'RT_cnt_wins_6_plus_L2G', 'Num_Players_3plus_ST_L4G_RT', 'Num_Players_10plus_TOT_L9G_RT', 'PPG_AVG_L8G_HT', 'TOT_AVG_L

In [105]:
initial_params['bagging_freq'] = 1

In [106]:
#Runs Hyperparameter tuning (Bayesian Optimization) - current scenario set to complete 200 rounds 
best_params, hyperparam_obj = model_scenario.run_hyperopt(model_scenario.hyperopt_param_space, 
                                                          model_scenario.X_vars, 
                                                          model_params = initial_params, 
                                                          fmin_max_evals = model_scenario.fmin_max_evals)

100%|██████████| 200/200 [09:38<00:00,  2.89s/trial, best loss: -0.5]


In [107]:
#Runs Hyperparameter tuning (Bayesian Optimization) - current scenario set to complete 200 rounds 
best_params

{'colsample_bytree': 0.6489325172213725,
 'learning_rate': 0.03265285941393413,
 'min_data_in_leaf': 100.0,
 'num_leaves': 49.0,
 'reg_alpha': 0.17948888280944775,
 'reg_lambda': 0.11267775892310361,
 'subsample': 0.9}

In [108]:
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['min_data_in_leaf'] = int(best_params['min_data_in_leaf'])

In [118]:
static_params =  {'objective': 'binary',
                  'metric': 'binary_logloss',
                  'num_boost_round': 1000,
                  'early_stopping_rounds': 25,
                  'max_depth': 15,
                  'seed': model_scenario.seed
                 }

In [119]:
final_model = lightgbm.LGBMModel(**best_params, **static_params, importance_type = 'gain')

In [120]:
eval_set = [(model_scenario.df_tune[list(post_rfe_model.booster_.feature_name())], 
             model_scenario.df_tune[model_scenario.target])]

In [121]:
final_model.fit(X = model_scenario.df_train[list(post_rfe_model.booster_.feature_name())],
                y = model_scenario.df_train[model_scenario.target],
                eval_set = eval_set,
                verbose = False)

LGBMModel(colsample_bytree=0.6489325172213725, early_stopping_rounds=25,
          importance_type='gain', learning_rate=0.03265285941393413,
          max_depth=15, metric='binary_logloss', min_data_in_leaf=100,
          num_boost_round=1000, num_leaves=49, objective='binary',
          reg_alpha=0.17948888280944775, reg_lambda=0.11267775892310361,
          seed=2454371, subsample=0.9)

In [135]:
model_predictions = final_model.predict(model_scenario.df_validate[post_rfe_model.booster_.feature_name()])

In [138]:
d = {
    'TEAM_HT' : model_scenario.df_validate['TEAM_HT'],
    'Final_Score_HT' : model_scenario.df_validate['Final_Score_HT'],
    'TEAM_RT' : model_scenario.df_validate['TEAM_RT'],
    'Final_Score_RT': model_scenario.df_validate['Final_Score_RT'],
    'outcome': model_scenario.df_validate['ht_spread_covered'],
    'ht_closing_spread': model_scenario.df_validate['CLOSING SPREAD_HT'],
    'model_prediction': model_predictions
}

In [139]:
model_prediction_df = pd.DataFrame(data = d)

In [142]:
model_prediction_df['model_predict'] = model_prediction_df['model_prediction'].apply(lambda x: 1 if x > 0.49 else 0)

In [143]:
(model_prediction_df['model_predict']  == model_prediction_df['outcome']).value_counts()

True     660
False    652
dtype: int64